In [119]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import re
import numpy as np
import itertools
import capstone2_utilities as cs2

# Data Cleaning
Collecting all the major steps I took to go from `Tweets.csv` from the [US Airline sentiment](https://www.kaggle.com/crowdflower/twitter-airline-sentiment#Tweets.csv) dataset available on Kaggle, to the file `tweets_with_originals.csv` that I use for sentiment analysis and the dataset data story. 

## Intro to the dataset
A look at the essential stats about our dataset, and a discussion of what possible issues were noticed during exploratory analysis. 

In [120]:
tweets = pd.read_csv('Tweets.csv', parse_dates = ['tweet_created'])
tweets.shape

(14640, 15)

In [121]:
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52-08:00,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59-08:00,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48-08:00,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36-08:00,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45-08:00,NaN,Pacific Time (US & Canada)


In [122]:
tweets.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [123]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
tweet_id                        14640 non-null int64
airline_sentiment               14640 non-null object
airline_sentiment_confidence    14640 non-null float64
negativereason                  9178 non-null object
negativereason_confidence       10522 non-null float64
airline                         14640 non-null object
airline_sentiment_gold          40 non-null object
name                            14640 non-null object
negativereason_gold             32 non-null object
retweet_count                   14640 non-null int64
text                            14640 non-null object
tweet_coord                     1019 non-null object
tweet_created                   14640 non-null datetime64[ns, pytz.FixedOffset(-480)]
tweet_location                  9907 non-null object
user_timezone                   9820 non-null object
dtypes: datetime64[ns, pytz.FixedOffset(-480)](1), float64(

We note that there aren't a lot of gold standard sentiments/ negative reasons. This isn't at all surprising, but just something to keep in mind when we draw conclusions at the end of the project. 

### HTML-general text
While looking at tweet lengths, I saw that some tweets were longer than 140 characters. These tweets contain HTML artifacts such as "&amp" and "&gt". We'll take care of these using the html parser from BeautifulSoup. 

In [124]:
tweets.loc[55].text

"@VirginAmerica hi! i'm so excited about your $99 LGA-&gt;DAL deal- but i've been trying 2 book since last week &amp; the page never loads. thx!"

In [125]:
from bs4 import BeautifulSoup
def parse_html(text):
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()
text_parsed=tweets.text.map(parse_html)
tweets['text'] = text_parsed

In [126]:
tweets.loc[55].text

"@VirginAmerica hi! i'm so excited about your $99 LGA->DAL deal- but i've been trying 2 book since last week & the page never loads. thx!"

### Duplicate tweets

In [127]:
no_appear = tweets.tweet_id.value_counts()
duplicate_tweets = no_appear.where(no_appear > 1).dropna()
df_duplicate = tweets[tweets.tweet_id.isin(duplicate_tweets.index)].sort_values(by='tweet_id')
duplicate_ids = list(duplicate_tweets.index)

In [128]:
duplicate_tweets.shape

(155,)

There are 155 duplicate tweets. It's a bit hard to tell what went on with these tweets but the sentiment confidence values can be different for the same tweet (in fact, they often are). 
What we care about the most are the positive/negative/neutral labels for airline sentiment. So for now, my solution is to aggregate the entries. For the sentiment_confidence and reason_confidence values, we'll take the mean. For negative reason we'll take a join so that we don't lose information. 

We'll refactor the duplicates, drop from tweets, and then concatenate the fixed entries. 

For the ones with two different sentiment labels we'll have to decide what to do:

In [129]:
no_different_sentiments = df_duplicate.groupby('tweet_id').airline_sentiment.apply(set).apply(len)
print(no_different_sentiments.where(no_different_sentiments > 1).dropna().shape)
drop_list = list(no_different_sentiments.where(no_different_sentiments > 1).dropna().index)

(18,)


Since there are only 18 tweets, we'll just drop them from consideration. 

For the rest of the tweets, we need to group them and aggregate the entries as planned. The utility function to do this, `combine_duplicate_tweets`, is in `capstone2_utilities`

In [130]:
df_duplicate = df_duplicate[~df_duplicate.tweet_id.isin(drop_list)]
refactored_duplicates = df_duplicate.groupby('tweet_id').apply(cs2.combine_duplicate_tweets)
refactored_duplicates = refactored_duplicates.droplevel(level=0).reset_index().drop('index', axis=1)

/home/rika/code/springboard/capstone2_nlp/capstone2_utilities.py:26: RuntimeWarning: Mean of empty slice
  row_dict['negativereason_confidence'] = np.nanmean(one_tweet_df['negativereason_confidence'])


In [131]:
len(refactored_duplicates.tweet_id.unique())

137

We merge the refactored duplicates back in to the dataframe and make sure that the tweet_ids are unique. 

In [132]:
tweets = tweets.loc[~tweets.tweet_id.isin(duplicate_ids)]
tweets = pd.concat([tweets, refactored_duplicates], axis = 0, ignore_index = True)
len(tweets.tweet_id.unique()) ==  tweets.tweet_id.shape[0]

True

### Recovering post-processed tweet text

An open question is how post-processed the `text` column of `Tweets.csv` is: 

In [133]:
tweets.text.iloc[18]

'I ❤️ flying @VirginAmerica. ☺️👍'

Emojis are definitely present. Further examination shows that emails also haven't been cleaned. Some of the emails are personal emails. 

In [134]:
# regex pattern to find emails in tweet text: 
#tweets.text.loc[tweets.text.str.contains('\w+@\w+')]
# an example of a (non-private) email identified using this regex pattern
tweets.text.loc[407]

"@VirginAmerica FYI the info@virginamerica.com email address you say to contact in password reset emails doesn't exist. Emails bounce."

While making text clouds of common words in the dataset (viewable in the data story notebook), I saw that there was indeed some post-processing on the tweet text. 

Words such as "late" have been replaced with "Late Flight", the stem "cancell-" has been replaced with "cancellation", etc. This appears to have been done to increase the performance of the "negative reason" classifier. 

The substitution also caught some undesired words: 

In [135]:
tweets.text.loc[tweets.text.str.contains('Late Flight')].loc[473]

'@VirginAmerica you got cheese pLate Flights too.'

In [136]:
tweets.iloc[129].text

'@VirginAmerica is flight 882 Cancelled Flightled and what do I do if it is?'

Since there is no further information about post-processing that took place on this dataset, I made the decision to try to obtain the original text of all tweets in the dataset. 

#### Step 1a: Retrieve original tweet text using Twitter API

In [137]:
import twitter
# CONSUMER_KEY, etc are private, you can get your own 
# if you sign up for a twitter developer acct. 

api = twitter.Api(consumer_key=CONSUMER_KEY,
                      consumer_secret=CONSUMER_SECRET,
                      access_token_key=ACCESS_TOKEN,
                      access_token_secret=ACCESS_TOKEN_SECRET, 
                      sleep_on_rate_limit=True)

In [138]:
original_tweet_text = tweets[['tweet_id']]
original_tweet_text = original_tweet_text.set_index('tweet_id')
original_tweet_text['text'] = ''

In [139]:
import time 
start_time = time.monotonic()
for rng in range(100, original_tweet_text.shape[0], 100):
    if rng % 1000 == 0: 
        print(rng, time.monotonic() - start_time)
    input_list = list(original_tweet_text.index[rng - 100: rng])
    request_out = api.GetStatuses(input_list)
    for status in request_out:
        original_tweet_text.loc[status.id, 'text'] = status.text
end_time = time.monotonic()
elapsed_time = end_time - start_time 
print(elapsed_time)
original_tweet_text.to_csv('original_tweets.csv')

1000 3.202491635000115
2000 6.152622059998976
3000 9.739212987999053
4000 12.830213305998768
5000 15.971789641000214
6000 19.47343714999988
7000 22.849111485998947
8000 26.751041147999786
9000 30.39045247699869
10000 33.783635535999565
11000 37.93341598999905
12000 41.816748117000316
13000 45.66926843100009
14000 50.219583870999486
52.21745352899961


In [140]:
original_tweet_text = pd.read_csv('original_tweets.csv')
tweets_with_originals = tweets.merge(original_tweet_text, on = 'tweet_id', suffixes = ['_proc', '_orig'])
missing_orig_tweets = tweets_with_originals[tweets_with_originals.text_orig.isna()][['tweet_id']]
missing_orig_tweets['text'] = ''
missing_orig_tweets = missing_orig_tweets.set_index('tweet_id')

This datasest was originally obtained in 2015. It's not surprising that many of the tweets are no longer available. 

In [141]:
tweets_with_originals[tweets_with_originals.text_orig.isna()].shape

(3196, 16)

#### Step 1b: Backsubstitute text for tweets that are no longer available. 
Since it looks like the text has not really been altered much otherwise, we'll backsubstitute the text in the rest of the tweets and regard this as "original text". 

It seems that the substitutions we see come from the negative reasons in the `negativereason` column. 

In [142]:
neg_reason_list = list(tweets_with_originals.negativereason.str.split(',').values)
neg_reason_list = [item for item in neg_reason_list if type(item) == list]
neg_reason_list = [item for sublist in neg_reason_list for item in sublist]
negative_reasons = set(item for item in neg_reason_list)

from collections import defaultdict
neg_reasons_dict = defaultdict(list)

Let's see which negative reasons really show up in the twitter text. 

We will assume that capitalization in this fashion comes from the replacement: of course, there is some risk that we're substituting perfectly accurate tweets: however, it's unlikely as social media users tend not to capitalize in this fashion. 

In [143]:
for elt in tweets_with_originals.loc[[30, 473, 129]].text_proc.values:
    print(elt)

@VirginAmerica hi! I just bked a cool birthday trip with you, but i can't add my elevate no. cause i entered my middle name during Flight Booking Problems 😢
@VirginAmerica you got cheese pLate Flights too.
@VirginAmerica is flight 882 Cancelled Flightled and what do I do if it is?


In [144]:
missing_orig = tweets_with_originals.loc[tweets_with_originals.text_orig.isna()].tweet_id.values
tweets_with_originals = tweets_with_originals.set_index('tweet_id')
for reason in negative_reasons: 
    reason_bool = tweets_with_originals.loc[missing_orig].text_proc.str.contains(reason)
    neg_reasons_dict[reason] = list(reason_bool.where(reason_bool == True).dropna().index)
[(i, len(neg_reasons_dict[i])) for i in neg_reasons_dict.keys() if len(neg_reasons_dict[i])!=0]

[('Cancelled Flight', 221),
 ('Flight Booking Problems', 28),
 (' Late Flight', 80),
 (' Flight Booking Problems', 18),
 (' Cancelled Flight', 220),
 ('Late Flight', 97)]

Looking at examples for each of the instances above, I built the following dict of replacements: 

In [145]:
backward_map = {'Cancelled Flight': 'cancell', 'Late Flight': 'late', 'Flight Booking Problems':'booking'}
tweets_with_originals.at[missing_orig, 'text_orig'] = tweets_with_originals.loc[missing_orig, 'text_proc']
for proc_word, orig_word in backward_map.items():
    tweets_with_originals.loc[missing_orig, 'text_orig'] = tweets_with_originals.loc[missing_orig,'text_orig'].str.replace(r'{}'.format(proc_word), r'{}'.format(orig_word), regex=True)
tweets_with_originals = tweets_with_originals.reset_index()

We confirm that it worked by checking our example tweets:

In [146]:
for elt in tweets_with_originals.loc[[30, 473, 129]].text_orig.values:
    print(elt)

@VirginAmerica hi! I just bked a cool birthday trip with you, but i can't add my elevate no. cause i entered my middle name during booking 😢
@VirginAmerica you got cheese plates too.
@VirginAmerica is flight 882 cancelled and what do I do if it is?


# Timezone Standardization

There are some nonstandard timezones in the dataset. The timezones allowed in Twitter come from Ruby on Rails `ActiveSupport::TimeZone`. Mapping to standardized timezones are [here](https://api.rubyonrails.org/classes/ActiveSupport/TimeZone.html)

In [147]:
timezone_dict = {}
with open('rails_timezones.txt', 'r') as file: 
    for line in file.readlines():
        nonstd, std = line.split('\t')
        timezone_dict[nonstd] = std.strip()

Some of the timezones ARE already standard, they should just remap to themselves. 

In [148]:
print(len(timezone_dict.keys()))
std_dict = {a:a for a in timezone_dict.values()}
timezone_dict.update(std_dict)
print(len(timezone_dict.keys()))

151
286


In [149]:
timezones = pd.DataFrame(tweets_with_originals.user_timezone)

In [150]:
timezones['remap'] = timezones.user_timezone.map(timezone_dict)

In [151]:
timezones.loc[~timezones.user_timezone.isna() & timezones.remap.isna()]

,user_timezone,remap
7529,America/Detroit,NaN
8375,America/Atikokan,NaN
11313,EST,NaN
13177,America/Boise,NaN
13555,America/Boise,NaN
14151,America/Boise,NaN


These 6 examples are hand-cleaned. The tweets are old, so perhaps they used to be allowed. 

In [152]:
timezones.at[[7529, 11313], 'remap'] = 'America/New_York'
timezones.at[[13177, 13555, 14151], 'remap'] = 'America/Denver' 
timezones.at[8375, 'remap'] = 'America/Chicago'

In [153]:
timezones.remap.value_counts().head()

America/New_York       3746
America/Chicago        1932
America/Los_Angeles    1211
America/Lima            725
America/Halifax         494
Name: remap, dtype: int64

In [154]:
tweets_with_originals['std_user_timezone'] = timezones['remap']

In [155]:
tweets_with_originals.to_csv('')

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text_proc,tweet_coord,tweet_created,tweet_location,user_timezone,text_orig,std_user_timezone
0,570306133677760513,neutral,1,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52-08:00,NaN,Eastern Time (US & Canada),@VirginAmerica What @dhepburn said.,America/New_York
1,570301130888122368,positive,0.3486,NaN,0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59-08:00,NaN,Pacific Time (US & Canada),@VirginAmerica plus you've added commercials t...,America/Los_Angeles
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48-08:00,Lets Play,Central Time (US & Canada),@VirginAmerica I didn't today... Must mean I n...,America/Chicago
3,570301031407624196,negative,1,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36-08:00,NaN,Pacific Time (US & Canada),@VirginAmerica it's really aggressive to blast...,America/Los_Angeles
4,570300817074462722,negative,1,Can't Tell,1,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45-08:00,NaN,Pacific Time (US & Canada),@VirginAmerica and it's a really big bad thing...,America/Los_Angeles
